In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('drive/MyDrive/faultSeg-master')

In [ ]:
import os
os.getcwd()

'/content/drive/MyDrive/faultSeg-master'

In [ ]:
from numpy.random import seed
seed(12345)
#from tensorflow import set_random_seed
import tensorflow
tensorflow.random.set_seed(1234)

#set_random_seed(1234)
import os
import random
import numpy as np
import skimage
import matplotlib.pyplot as plt
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau, TensorBoard
from keras import backend as keras
from utils import DataGenerator
from keras.utils import DataGenerator
from tensorflow.keras.utils import DataGenerator
from unet3 import *
import datetime


In [ ]:
import tensorflow as tf
tf.__version__

'2.12.0'

In [5]:
def showHistory(history):
  # list all data in history
  print(history.history.keys())
  fig = plt.figure(figsize=(10,6))

  # summarize history for accuracy
  plt.plot(history.history['acc'])
  plt.plot(history.history['val_acc'])
  plt.title('Model accuracy',fontsize=20)
  plt.ylabel('Accuracy',fontsize=20)
  plt.xlabel('Epoch',fontsize=20)
  plt.legend(['train', 'test'], loc='center right',fontsize=20)
  plt.tick_params(axis='both', which='major', labelsize=18)
  plt.tick_params(axis='both', which='minor', labelsize=18)
  plt.show()

  # summarize history for loss
  fig = plt.figure(figsize=(10,6))
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('Model loss',fontsize=20)
  plt.ylabel('Loss',fontsize=20)
  plt.xlabel('Epoch',fontsize=20)
  plt.legend(['train', 'test'], loc='center right',fontsize=20)
  plt.tick_params(axis='both', which='major', labelsize=18)
  plt.tick_params(axis='both', which='minor', labelsize=18)
  plt.show()

In [6]:
class TrainValTensorBoard(TensorBoard):
    def __init__(self, log_dir='./log1', **kwargs):
        # Make the original `TensorBoard` log to a subdirectory 'training'
        training_log_dir = os.path.join(log_dir, 'training')
        super(TrainValTensorBoard, self).__init__(training_log_dir, **kwargs)
        # Log the validation metrics to a separate subdirectory
        self.val_log_dir = os.path.join(log_dir, 'validation')
    def set_model(self, model):
        # Setup writer for validation metrics
        #self.val_writer = tf.summary.FileWriter(self.val_log_dir)
        self.val_writer = tf.summary.create_file_writer(self.val_log_dir)
        super(TrainValTensorBoard, self).set_model(model)
    def on_epoch_end(self, epoch, logs=None):
        # Pop the validation logs and handle them separately with
        # `self.val_writer`. Also rename the keys so that they can
        # be plotted on the same figure with the training metrics
        logs = logs or {}
        val_logs = {k.replace('val_', ''): v for k, v in logs.items() if k.startswith('val_')}
        for name, value in val_logs.items():
            summary = tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = value.item()
            summary_value.tag = name
            self.val_writer.add_summary(summary, epoch)
        self.val_writer.flush()
        # Pass the remaining logs to `TensorBoard.on_epoch_end`
        logs = {k: v for k, v in logs.items() if not k.startswith('val_')}
        logs.update({'lr': keras.eval(self.model.optimizer.lr)})
        super(TrainValTensorBoard, self).on_epoch_end(epoch, logs)
    def on_train_end(self, logs=None):
        super(TrainValTensorBoard, self).on_train_end(logs)
        self.val_writer.close()

In [ ]:
def main():
  goTrain()

def goTrain():
  # input image dimensions
    params = {'batch_size':1,
          'dim':(128,128,128),
          'n_channels':1,
          'shuffle': True}

    BATCH_SIZE = 1
    STEPS_PER_EPOCH = 100 / BATCH_SIZE
    SAVE_PERIOD = 10


    logdir = "./logtf/"
    seismPathT = "./data/train/seis/"
    faultPathT = "./data/train/fault/"

    seismPathV = "./data/validation/seis/"
    faultPathV = "./data/validation/fault/"
    train_ID = range(100)
    valid_ID = range(20)
    train_generator = DataGenerator(dpath=seismPathT,fpath=faultPathT,
                                  data_IDs=train_ID,**params)
    valid_generator = DataGenerator(dpath=seismPathV,fpath=faultPathV,
                                  data_IDs=valid_ID,**params)
    model = unet(input_size=(None, None, None,1))
    model.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy',
                metrics=['accuracy'])
    model.summary()



# checkpoint
    filepath="./check1/fseg-{epoch:02d}.hdf5"
    # checkpoint = ModelCheckpoint(filepath, monitor='val_acc',
    #     verbose=1, save_best_only=True, mode='max',save_freq=10)

    cp_callback = ModelCheckpoint(
    filepath= filepath,
    verbose=1,
    save_weights_only=True,
    save_freq= int(SAVE_PERIOD * STEPS_PER_EPOCH))

    print("data prepared, ready to train!")
    # Fit the model
    tensorboard_callback = tensorflow.keras.callbacks.TensorBoard(log_dir=logdir,histogram_freq=0,
          write_graph=True, write_images=True)
    history=model.fit_generator(generator=train_generator,
    validation_data=valid_generator,epochs=100,callbacks=[tensorboard_callback, cp_callback],verbose=1)

    print("Average test loss: ", np.average(history.history['loss']))


In [7]:
if __name__ == '__main__':
    main()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 None, 1)]                                                        
                                                                                                  
 conv3d (Conv3D)                (None, None, None,   448         ['input_1[0][0]']                
                                None, 16)                                                         
                                                                                                  
 conv3d_1 (Conv3D)              (None, None, None,   6928        ['conv3d[0][0]']                 
                                None, 16)                                                     

 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 None, 1)]                                                        
                                                                                                  
 conv3d (Conv3D)                (None, None, None,   448         ['input_1[0][0]']                
                                None, 16)                                                         
                                                                                                  
 conv3d_1 (Conv3D)              (None, None, None,   6928        ['conv3d[0][0]']                 
                                None, 16)                                                         
                                                                                                  
 max_pooling3d (MaxPooling3D)   (None, None, None,   0           ['conv3d_1[0][0]']               
          

E:\Software\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
C:\Users\DELL\AppData\Local\Temp\ipykernel_5004\3113077462.py:39: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history=model.fit_generator(generator=train_generator,


Epoch 1/100
 10/100 [==>...........................] - ETA: 12:23:00 - loss: 0.6691 - accuracy: 0.7184

ResourceExhaustedError: Graph execution error:

Detected at node 'gradient_tape/model/conv3d_12/Conv3D/Conv3DBackpropFilterV2' defined at (most recent call last):
    File "E:\Software\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "E:\Software\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "E:\Software\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "E:\Software\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "E:\Software\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "E:\Software\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "E:\Software\lib\asyncio\base_events.py", line 601, in run_forever
      self._run_once()
    File "E:\Software\lib\asyncio\base_events.py", lin